In [1]:
import torch
import os
from torchvision import transforms

In [ ]:
train_dir = os.path.join('dataset', 'part_one_dataset', 'train_data')
eval_dir = os.path.join('dataset', 'part_one_dataset', 'eval_data')
save_dir = os.path.join('vit')

In [ ]:
domains = [{} for _ in range(10)]

for j in range(10):
    
    train_path = os.path.join(train_dir, f'{j+1}_train_data.tar.pth')
    t = torch.load(train_path, weights_only = False)
    
    domains[j]['labels'] = t['targets'] if 'targets' in t else None
    domains[j]['features'] = torch.load(os.path.join(save_dir,f'train_embeds_{j+1}.pt'))

/tmp/ipykernel_171887/3573861280.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  domains[j]['features'] = torch.load(f'stuff/train_embeds_{j+1}.pt')


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_distances, manhattan_distances

class LWP:
    """Learning Vector Prototypes with configurable distance function"""
    
    DISTANCE_FUNCTIONS = {
        'euclidean': lambda x, y: np.linalg.norm(x - y),
        'cosine': lambda x, y: cosine_distances(x.reshape(1, -1), y.reshape(1, -1))[0][0],
        'manhattan': lambda x, y: manhattan_distances(x.reshape(1, -1), y.reshape(1, -1))[0][0],
        'minkowski': lambda x, y, p=2: np.power(np.sum(np.power(np.abs(x - y), p)), 1/p)
    }
    
    def __init__(self, distance_metric='euclidean', **distance_params):
        """
            distance_params (dict): Additional parameters for the distance function
        """
        self.prototypes = {}
        self.class_counts = {i: 0 for i in range(10)}
        
        if callable(distance_metric):
            self.distance_fn = distance_metric
        elif distance_metric in self.DISTANCE_FUNCTIONS:
            if distance_metric == 'minkowski':
                p = distance_params.get('p', 2)
                self.distance_fn = lambda x, y: self.DISTANCE_FUNCTIONS[distance_metric](x, y, p)
            else:
                self.distance_fn = self.DISTANCE_FUNCTIONS[distance_metric]
        else:
            raise ValueError(f"Unknown distance metric: {distance_metric}. " 
                           f"Available metrics: {list(self.DISTANCE_FUNCTIONS.keys())}")

    def fit(self, features, labels):
        unique_labels = np.unique(labels)
        for label in unique_labels:
            samples = features[labels == label]
            num_samples = len(samples)
            
            if label not in self.prototypes:  # Original condition was: if label not in self.prototypes
                self.prototypes[label] = samples.mean(axis=0)
                self.class_counts[label] = len(samples)
            else:
                self.class_counts[label] += len(samples)
                self.prototypes[label] = (
                    (self.class_counts[label] - num_samples) / self.class_counts[label] * self.prototypes[label] +
                    num_samples / self.class_counts[label] * samples.mean(axis=0)
                )

    def predict(self, features):
        preds = []
        for feature in features:
            distances = {
                label: self.distance_fn(feature, proto)
                for label, proto in self.prototypes.items()
            }
            preds.append(min(distances, key=distances.get))
        return np.array(preds)

## Get Eval Data
Evaluating on trainset for now

In [ ]:
eval_domains = [{} for _ in range(10)]

for j in range(10):
    
    eval_path = os.path.join(eval_dir, f'{j+1}_eval_data.tar.pth')
    t = torch.load(eval_path, weights_only = False)

    data = t['data'] # both numpy.ndarray
    
    eval_domains[j]['labels'] = t['targets'] if 'targets' in t else None
    eval_domains[j]['features'] = torch.load(os.path.join(save_dir,f'eval_embeds_{j+1}.pt'))

/tmp/ipykernel_171887/487064672.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  eval_domains[j]['features'] = torch.load(f'stuff/eval_embeds_{j+1}.pt')


In [10]:
from sklearn.metrics import accuracy_score
import pandas as pd

model = LWP(distance_metric='cosine')

df = pd.DataFrame()

for idx,domain in enumerate(domains):
    
    x_test = domain['features']
    y_pred = model.predict(x_test) if domain['labels'] is None else domain['labels']
    
    model.fit(x_test, y_pred)
    print(model.class_counts)
    
    scores = []
    for eval_domain in eval_domains[:idx+1]:
        
        features = eval_domain['features']
        labels = eval_domain['labels']
        
        preds = model.predict(features)
        acc = accuracy_score(labels, preds)
        
        scores.append(acc)
    
    df[f'Domain {idx+1}'] = scores + [np.nan] * (len(eval_domains) - len(scores))

{0: 253, 1: 243, 2: 255, 3: 244, 4: 262, 5: 236, 6: 250, 7: 253, 8: 254, 9: 250}
{0: 163, 1: 315, 2: 247, 3: 54, 4: 334, 5: 119, 6: 329, 7: 213, 8: 248, 9: 478}
{0: 144, 1: 302, 2: 245, 3: 209, 4: 285, 5: 149, 6: 311, 7: 230, 8: 251, 9: 374}
{0: 130, 1: 302, 2: 235, 3: 295, 4: 208, 5: 210, 6: 307, 7: 218, 8: 217, 9: 378}
{0: 138, 1: 285, 2: 274, 3: 325, 4: 188, 5: 228, 6: 294, 7: 204, 8: 214, 9: 350}
{0: 174, 1: 314, 2: 261, 3: 335, 4: 205, 5: 229, 6: 287, 7: 181, 8: 240, 9: 274}
{0: 166, 1: 286, 2: 247, 3: 313, 4: 221, 5: 253, 6: 298, 7: 178, 8: 267, 9: 271}
{0: 185, 1: 280, 2: 242, 3: 339, 4: 209, 5: 260, 6: 319, 7: 143, 8: 220, 9: 303}
{0: 219, 1: 256, 2: 246, 3: 357, 4: 253, 5: 213, 6: 295, 7: 205, 8: 221, 9: 235}
{0: 274, 1: 271, 2: 250, 3: 303, 4: 221, 5: 231, 6: 302, 7: 188, 8: 200, 9: 260}


In [12]:
print(df)

   Domain 1  Domain 2  Domain 3  Domain 4  Domain 5  Domain 6  Domain 7  \
0    0.2684    0.2436    0.2360    0.2232    0.2120    0.2032    0.2032   
1       NaN    0.2404    0.2252    0.2172    0.2156    0.2052    0.1992   
2       NaN       NaN    0.2340    0.2276    0.2180    0.2108    0.2100   
3       NaN       NaN       NaN    0.2304    0.2224    0.2212    0.2212   
4       NaN       NaN       NaN       NaN    0.2232    0.2212    0.2212   
5       NaN       NaN       NaN       NaN       NaN    0.1988    0.1964   
6       NaN       NaN       NaN       NaN       NaN       NaN    0.2120   
7       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
8       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
9       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

   Domain 8  Domain 9  Domain 10  
0    0.2012    0.1976     0.1988  
1    0.1948    0.1956     0.1932  
2    0.2020    0.1968     0.1932  
3    0.2152    0.2132     0.2056  